In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import lightgbm as lgb

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
df = pd.read_csv("../data/훈련데이터셋/3_훈련데이터셋_LAG_휴일포함.csv")
#df.info()

In [3]:
df.tail(5)

,날짜,전체코드,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워,lag_1d,lag_7d,휴일여부
233203,2024-01-24,1174011000,-11.2,2.3,1.7,10.1,52.0,115124.547,114960.859,108004.536,False
233204,2024-01-25,1174011000,-9.7,2.2,2.0,10.1,47.8,99907.500,115124.547,98758.864,False
233205,2024-01-26,1174011000,-6.9,2.1,3.4,10.1,62.5,94818.512,99907.500,NaN,False
233206,2024-01-27,1174011000,-4.6,2.0,4.0,10.2,70.0,86792.828,94818.512,76691.460,True
233207,2024-01-28,1174011000,-5.0,1.9,3.4,10.2,62.3,86389.477,86792.828,89640.934,True


In [4]:
df["휴일여부"] = df["휴일여부"].astype(int)

In [5]:
feature_cols = [
    # 지역
    "전체코드",

    # 날씨
    
    "최저기온(°C)", "0.5m 지중온도(°C)", "평균 증기압(hPa)","가조시간(hr)","평균 상대습도(%)",

    # 기간

    "lag_1d", "lag_7d"

]
target_col = "파워"

In [6]:
feature_cols = feature_cols + ["휴일여부"]

In [8]:
# new_df = df.dropna(subset=["파워"]).reset_index(drop=True)

In [29]:
new_df = df[["날짜"] + feature_cols + [target_col]].dropna().reset_index(drop=True)

In [10]:
# print(len(df), len(new_df))
# new_df.isna().sum().sort_values(ascending=False)

In [11]:
# =========================
# 1) 시간순 Train/Valid/Test 분할
# =========================
new_df = new_df.sort_values("날짜").reset_index(drop=True)

n = len(new_df)
test_size = int(n * 0.20)
valid_size = int(n * 0.10)

train_end = n - (valid_size + test_size)
valid_end = n - test_size

train_df = new_df.iloc[:train_end]
valid_df = new_df.iloc[train_end:valid_end]
test_df  = new_df.iloc[valid_end:]

In [12]:
# =========================
# 2) X / y 분리
# =========================
drop_cols = [target_col, "날짜"]  # 날짜는 직접 입력 피처로 쓰지 않음(우린 파생변수로 이미 반영)
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_col]

In [13]:

# =========================
# 3) 범주형 처리 (코드형 컬럼은 category로)
# =========================
cat_cols = []
for c in ["전체코드", "시군구코드", "법정동코드"]:
    if c in X_train.columns:
        cat_cols.append(c)

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_valid[c] = X_valid[c].astype("category")
    X_test[c]  = X_test[c].astype("category")

In [14]:
# =========================
# 4) LightGBM Baseline 모델
# =========================
model = LGBMRegressor(
    objective="regression_l2",   # 🔥 반드시 변경

    n_estimators=20000,
    learning_rate=0.03,

    num_leaves=63,
    max_depth=10,
    min_child_samples=80,        # ↓ 완화
    min_split_gain=0.05,         # ↓ 완화

    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,

    reg_alpha=0.1,               # ↓
    reg_lambda=5.0,              # ↓

    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
    callbacks=[
        # early stopping (너무 오래 돌지 않게)
        # verbose는 200번마다 한번씩 로그
        __import__("lightgbm").early_stopping(stopping_rounds=200, verbose=True),
        __import__("lightgbm").log_evaluation(period=200),
    ]
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003098 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1787
[LightGBM] [Info] Number of data points in the train set: 106549, number of used features: 9
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 141294.244140
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, max_depth=10,
              min_child_samples=80, min_split_gain=0.05, n_estimators=20000,
              n_jobs=-1, num_leaves=63, objective='regression_l2',
              random_state=42, reg_alpha=0.1, reg_lambda=5.0, subsample=0.8,
              subsample_freq=1)

In [15]:
# =========================
# 5) 평가 (Valid / Test)
# =========================
def eval_print(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE: {rmse:,.3f}")
    print(f"[{name}] R2  : {r2:.6f}")
    return rmse, r2

pred_valid = model.predict(X_valid)
pred_test  = model.predict(X_test)

eval_print("VALID", y_valid, pred_valid)
eval_print("TEST",  y_test,  pred_test)

print("Best iteration:", model.best_iteration_)

[VALID] RMSE: 188,588.899
[VALID] R2  : 0.756616
[TEST] RMSE: 89,264.184
[TEST] R2  : 0.841275
Best iteration: 830


In [16]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =========================
# 예측
# =========================
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
y_test_pred  = model.predict(X_test)

# =========================
# MSE / MAE 계산
# =========================
metrics = {
    "TRAIN": {
        "MSE": mean_squared_error(y_train, y_train_pred),
        "MAE": mean_absolute_error(y_train, y_train_pred)
    },
    "VALID": {
        "MSE": mean_squared_error(y_valid, y_valid_pred),
        "MAE": mean_absolute_error(y_valid, y_valid_pred)
    },
    "TEST": {
        "MSE": mean_squared_error(y_test, y_test_pred),
        "MAE": mean_absolute_error(y_test, y_test_pred)
    }
}

# =========================
# 출력
# =========================
for k, v in metrics.items():
    print(f"[{k}]")
    print(f"  MSE : {v['MSE']:,.3f}")
    print(f"  MAE : {v['MAE']:,.3f}")

[TRAIN]
  MSE : 7,500,562,051.399
  MAE : 9,033.484
[VALID]
  MSE : 35,565,772,698.204
  MAE : 13,606.637
[TEST]
  MSE : 7,968,094,463.152
  MAE : 12,271.235


In [17]:
# 전체 중요도



In [18]:
# 중요도 전체

import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

imp.head(20)

,feature,importance
0,전체코드,6552
6,lag_1d,6123
7,lag_7d,5824
1,최저기온(°C),4106
2,0.5m 지중온도(°C),4027
3,평균 증기압(hPa),3829
5,평균 상대습도(%),3689
4,가조시간(hr),3019
8,휴일여부,1289


In [26]:
# 전체 중요도
import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

imp["importance_pct"] = imp["importance"] / imp["importance"].sum() * 100

imp.head(20)

,feature,importance,importance_pct
0,전체코드,6552,17.036767
6,lag_1d,6123,15.921265
7,lag_7d,5824,15.143793
1,최저기온(°C),4106,10.676582
2,0.5m 지중온도(°C),4027,10.471163
3,평균 증기압(hPa),3829,9.956316
5,평균 상대습도(%),3689,9.592282
4,가조시간(hr),3019,7.850122
8,휴일여부,1289,3.351708


In [56]:
df["전체코드"].unique()[:77]

array([1111010100, 1111010200, 1111010300, 1111010400, 1111010500,
       1111010600, 1111010700, 1111010800, 1111010900, 1111011000,
       1111011100, 1111011200, 1111011300, 1111011400, 1111011500,
       1111011600, 1111011700, 1111011800, 1111011900, 1111012000,
       1111012100, 1111012200, 1111012300, 1111012400, 1111012500,
       1111012600, 1111012700, 1111012800, 1111012900, 1111013000,
       1111013100, 1111013200, 1111013300, 1111013400, 1111013500,
       1111013600, 1111013700, 1111013800, 1111013900, 1111014000,
       1111014100, 1111014200, 1111014300, 1111014400, 1111014500,
       1111014600, 1111014700, 1111014800, 1111014900, 1111015000,
       1111015100, 1111015200, 1111015300, 1111015400, 1111015500,
       1111015600, 1111015700, 1111015800, 1111015900, 1111016000,
       1111016100, 1111016200, 1111016300, 1111016400, 1111016500,
       1111016600, 1111016700, 1111016800, 1111016900, 1111017000,
       1111017100, 1111017200, 1111017300, 1111017400, 1111017

In [58]:
import pandas as pd

# 🔹 직접 입력값
input_data = {
    '전체코드': 1111017300,        # 동 코드
    '최저기온(°C)': -3.2,
    '0.5m 지중온도(°C)': 1.5,
    '평균 증기압(hPa)': 6.8,
    '가조시간(hr)': 5.2,
    '평균 상대습도(%)': 62.0,
    'lag_1d': 123456.0,            # 어제 전력
    'lag_7d': 164400.0,            # 7일 전 전력
    '휴일여부': 0                  # 평일
}



In [59]:
cat_cols = X_train.select_dtypes(include="category").columns

In [60]:
X_input = pd.DataFrame([input_data])[feature_cols]

# 🔹 categorical 컬럼 처리 (핵심)
for col in cat_cols:
    X_input[col] = X_input[col].astype("category")
    X_input[col] = X_input[col].cat.set_categories(
        X_train[col].cat.categories
    )

In [61]:
pred = model.predict(X_input)
print(f"예측 전력 사용량: {pred[0]:,.2f}")

예측 전력 사용량: 142,913.61
